<a href="https://colab.research.google.com/github/nghiaphan177/Truyvanttdpt/blob/main/Feature_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pickle
import pandas as pd 
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import numpy as np

In [2]:
with open('News_dataset.pickle', 'rb') as data:
    df = pickle.load(data)

In [3]:
df.head()

,Unnamed: 0,content,category,id,News_length
0,0,"\nThuở hàn vi, các hãng mua máy bay chỉ bằng c...",kinh-doanh,1,13110
1,1,\nGiá vàng trong nước sáng nay tiếp tục giảm t...,kinh-doanh,1,1164
2,2,"\nSo với lúc VN-Index tạo đáy cuối tháng 3, 29...",kinh-doanh,1,4345
3,3,"\nTrong bối cảnh Covid-19, nhiều người không p...",kinh-doanh,1,4188
4,4,\nViệt Nam có thể là một trong những quốc gia ...,kinh-doanh,1,8242


In [4]:
df.loc[4]['content']

'\nViệt Nam có thể là một trong những quốc gia nhận nhiều lợi ích từ RCEP khi có các mặt hàng thế mạnh như nông, thuỷ sản đáp ứng nhu cầu hầu hết thành viên.\nSau 8 năm đàm phán, Hiệp định Đối tác Kinh tế toàn diện khu vực (RCEP) đã được 15 nước, bao gồm: Brunei, Malaysia, Singapore, Việt Nam, Campuchia, Indonesia, Lào, Myanmar, Philippines, Thái Lan, Nhật Bản, New Zealand, Australia, Trung Quốc, Hàn Quốc, ký kết sáng 15/11.\n"Trong khi Covid-19 làm gián đoạn chuỗi cung ứng toàn cầu, xu hướng bảo hộ đang nổi lên, RCEP được ký kết đánh dấu mốc quan trọng trong hội nhập kinh tế của các nước tham gia đàm phán", Bộ trưởng Công Thương Trần Tuấn Anh nhấn mạnh sau buổi lễ.\nRCEP, với sự tham gia của 15 thành viên, sẽ tạo ra thị trường trên quy mô 2,2 tỷ người, tương đương 26.200 tỷ USD, tạo nên khu vực thuơng mại tự do lớn nhất thế giới. Nhờ vào cam kết mở cửa thị trường hàng hoá, dịch vụ, đầu tư, quy tắc xuất xứ giá trị trong khu vực địa lý RCEP cùng các biện pháp tạo thuận lợi thương mại, F

## 1. Text cleaning and preparation

 **1.1. Remove unnecssary character**

In [5]:
df['content_parsed_1'] = df['content'].replace(regex=r'[^\d\w ]', value='')

 **1.2. downcase**

In [6]:
df['content_parsed_2'] = df['content_parsed_1'].str.lower()

In [7]:
df.head()

,Unnamed: 0,content,category,id,News_length,content_parsed_1,content_parsed_2
0,0,"\nThuở hàn vi, các hãng mua máy bay chỉ bằng c...",kinh-doanh,1,13110,Thuở hàn vi các hãng mua máy bay chỉ bằng cách...,thuở hàn vi các hãng mua máy bay chỉ bằng cách...
1,1,\nGiá vàng trong nước sáng nay tiếp tục giảm t...,kinh-doanh,1,1164,Giá vàng trong nước sáng nay tiếp tục giảm the...,giá vàng trong nước sáng nay tiếp tục giảm the...
2,2,"\nSo với lúc VN-Index tạo đáy cuối tháng 3, 29...",kinh-doanh,1,4345,So với lúc VNIndex tạo đáy cuối tháng 3 2930 m...,so với lúc vnindex tạo đáy cuối tháng 3 2930 m...
3,3,"\nTrong bối cảnh Covid-19, nhiều người không p...",kinh-doanh,1,4188,Trong bối cảnh Covid19 nhiều người không phải ...,trong bối cảnh covid19 nhiều người không phải ...
4,4,\nViệt Nam có thể là một trong những quốc gia ...,kinh-doanh,1,8242,Việt Nam có thể là một trong những quốc gia nh...,việt nam có thể là một trong những quốc gia nh...


In [8]:
list_columns = ['content', 'category', 'content_parsed_2']
df = df[list_columns]

df = df.rename(columns={'content_parsed_2' : 'content_parsed'})

In [9]:
df.head()

,content,category,content_parsed
0,"\nThuở hàn vi, các hãng mua máy bay chỉ bằng c...",kinh-doanh,thuở hàn vi các hãng mua máy bay chỉ bằng cách...
1,\nGiá vàng trong nước sáng nay tiếp tục giảm t...,kinh-doanh,giá vàng trong nước sáng nay tiếp tục giảm the...
2,"\nSo với lúc VN-Index tạo đáy cuối tháng 3, 29...",kinh-doanh,so với lúc vnindex tạo đáy cuối tháng 3 2930 m...
3,"\nTrong bối cảnh Covid-19, nhiều người không p...",kinh-doanh,trong bối cảnh covid19 nhiều người không phải ...
4,\nViệt Nam có thể là một trong những quốc gia ...,kinh-doanh,việt nam có thể là một trong những quốc gia nh...


## 2. Label encoding

In [10]:
category_codes = {'giai-tri' : 1,
                  'phap-luat' : 2,
                  'kinh-doanh' : 3}

In [11]:
df['category_code'] = df['category']
df = df.replace({'category_code' : category_codes})

In [12]:
df.head()

,content,category,content_parsed,category_code
0,"\nThuở hàn vi, các hãng mua máy bay chỉ bằng c...",kinh-doanh,thuở hàn vi các hãng mua máy bay chỉ bằng cách...,3
1,\nGiá vàng trong nước sáng nay tiếp tục giảm t...,kinh-doanh,giá vàng trong nước sáng nay tiếp tục giảm the...,3
2,"\nSo với lúc VN-Index tạo đáy cuối tháng 3, 29...",kinh-doanh,so với lúc vnindex tạo đáy cuối tháng 3 2930 m...,3
3,"\nTrong bối cảnh Covid-19, nhiều người không p...",kinh-doanh,trong bối cảnh covid19 nhiều người không phải ...,3
4,\nViệt Nam có thể là một trong những quốc gia ...,kinh-doanh,việt nam có thể là một trong những quốc gia nh...,3


## Train - test split

In [13]:
X_train, X_test, Y_train, Y_test = train_test_split(df['content_parsed'], 
                                                    df['category_code'], 
                                                    test_size=0.15, 
                                                    random_state=8)

## Text representation

In [14]:
# Parameter election
ngram_range = (1,2)
min_df = 10
max_df = 1.
max_features = 300  

In [15]:
tfidf = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        max_features=max_features,
                        norm='l2',
                        sublinear_tf=True)
                        
features_train = tfidf.fit_transform(X_train).toarray()
labels_train = Y_train
print(features_train.shape)

features_test = tfidf.transform(X_test).toarray()
labels_test = Y_test
print(features_test.shape)

(1275, 300)
(225, 300)


## Save to file

In [16]:
import os

dir = os.path.join('/content', 'Pickles')

if not os.path.exists(dir):
  os.mkdir(dir)


In [17]:
# X_train
with open('Pickles/X_train.pickle', 'wb') as output:
    pickle.dump(X_train, output)
    
# X_test    
with open('Pickles/X_test.pickle', 'wb') as output:
    pickle.dump(X_test, output)
    
# y_train
with open('Pickles/Y_train.pickle', 'wb') as output:
    pickle.dump(Y_train, output)
    
# y_test
with open('Pickles/Y_test.pickle', 'wb') as output:
    pickle.dump(Y_test, output)
    
# df
with open('Pickles/df.pickle', 'wb') as output:
    pickle.dump(df, output)
    
# features_train
with open('Pickles/features_train.pickle', 'wb') as output:
    pickle.dump(features_train, output)

# labels_train
with open('Pickles/labels_train.pickle', 'wb') as output:
    pickle.dump(labels_train, output)

# features_test
with open('Pickles/features_test.pickle', 'wb') as output:
    pickle.dump(features_test, output)

# labels_test
with open('Pickles/labels_test.pickle', 'wb') as output:
    pickle.dump(labels_test, output)
    
# TF-IDF object
with open('Pickles/tfidf.pickle', 'wb') as output:
    pickle.dump(tfidf, output)

In [18]:
!zip Pickles.zip Pickles

  adding: Pickles/ (stored 0%)


In [19]:
!zip -r Pickles.zip Pickles

updating: Pickles/ (stored 0%)
  adding: Pickles/X_test.pickle (deflated 69%)
  adding: Pickles/Y_test.pickle (deflated 79%)
  adding: Pickles/features_test.pickle (deflated 61%)
  adding: Pickles/df.pickle (deflated 69%)
  adding: Pickles/X_train.pickle (deflated 69%)
  adding: Pickles/tfidf.pickle (deflated 58%)
  adding: Pickles/Y_train.pickle (deflated 86%)
  adding: Pickles/features_train.pickle (deflated 62%)
  adding: Pickles/labels_test.pickle (deflated 79%)
  adding: Pickles/labels_train.pickle (deflated 86%)
